In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

# Standard libs
import os, re, base64
import numpy as np
import pandas as pd

# CRUD module (Project One)
from AnimalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "Wanda1122#"

# Connect to database via CRUD Module
db = AnimalShelter(
    username,
    password,
    host='nv-desktop-services.apporto.com',
    port=34195,
    db='AAC',          # <-- uppercase matches mongoimport
    col='animals',
    auth_source='AAC'  # <-- new param
)


# Helper: get full dataset as a DataFrame (drop _id safely)
def read_df(query=None, projection=None):
    query = query or {}
    recs = db.read(query, projection) if projection is not None else db.read(query)
    df = pd.DataFrame.from_records(recs)
    if "_id" in df.columns:
        df.drop(columns=["_id"], inplace=True)
    return df

# --- DEFINE q_reset() BEFORE IT'S USED ---
def q_reset():
    # Show all dogs (case-insensitive)
    return {"animal_type": {"$regex": "^Dog$", "$options": "i"}}

# Initial data = all dogs so the UI isn't empty on first render
df = read_df(q_reset())
print("Initial load (q_reset) rows:", len(df))

# If the DB is empty, keep a minimal placeholder to avoid crashes
if df.empty:
    df = read_df({})  # try ALL documents so you can see something while debugging
    print("Fallback ALL-docs rows:", len(df))
    if df.empty:
        df = pd.DataFrame([{
            "animal_type":"Dog",
            "breed":"Unknown",
            "sex_upon_outcome":"Unknown",
            "age_upon_outcome_in_weeks":0,
            "name":"(no data)",
            "outcome_type":"Unknown",
            "location_lat":30.75,
            "location_long":-97.48
        }])


#########################
# Column Name Normalizer
#########################
COLUMN_ALIASES = {
    "lat": ["location_lat", "latitude", "lat"],
    "lon": ["location_long", "longitude", "lon"],
    "breed": ["breed", "animal_breed"],
    "name": ["name", "animal_name"],
    "sex": ["sex_upon_outcome", "sex"],
    "age_weeks": ["age_upon_outcome_in_weeks", "age_weeks"]
}

def first_existing(col_list, cols):
    for c in col_list:
        if c in cols: return c
    return None

def resolve_cols(df):
    cols = df.columns
    return {
        "lat": first_existing(COLUMN_ALIASES["lat"], cols),
        "lon": first_existing(COLUMN_ALIASES["lon"], cols),
        "breed": first_existing(COLUMN_ALIASES["breed"], cols),
        "name": first_existing(COLUMN_ALIASES["name"], cols),
        "sex": first_existing(COLUMN_ALIASES["sex"], cols),
        "age_weeks": first_existing(COLUMN_ALIASES["age_weeks"], cols),
    }

RESOLVED = resolve_cols(df)


#########################
# Rescue Query Builders
#########################
def regex_any_contains(words):
    pattern = "(" + "|".join([re.sub(r"\s+", r"\\s*", w) for w in words]) + ")"
    return {"$regex": pattern, "$options": "i"}

def q_water(strict=True):
    base = {
        "animal_type": {"$regex": "^Dog$", "$options": "i"},
        "breed": regex_any_contains(["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"]),
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    }
    base["sex_upon_outcome"] = {"$regex": r"^Intact\s*Female$", "$options": "i"} if strict \
                               else {"$regex": r"(Intact\s*Female|Spayed\s*Female)", "$options": "i"}
    return base

def q_mountain(strict=True):
    base = {
        "animal_type": {"$regex": "^Dog$", "$options": "i"},
        "breed": regex_any_contains(["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]),
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    }
    base["sex_upon_outcome"] = {"$regex": r"^Intact\s*Male$", "$options": "i"} if strict \
                               else {"$regex": r"(Intact\s*Male|Neutered\s*Male)", "$options": "i"}
    return base

def q_disaster(strict=True):
    base = {
        "animal_type": {"$regex": "^Dog$", "$options": "i"},
        "breed": regex_any_contains(["Doberman", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]),
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    }
    base["sex_upon_outcome"] = {"$regex": r"^Intact\s*Male$", "$options": "i"} if strict \
                               else {"$regex": r"(Intact\s*Male|Neutered\s*Male)", "$options": "i"}
    return base

def build_query(filter_type, strict=True):
    return {
        "water": lambda: q_water(strict),
        "mountain": lambda: q_mountain(strict),
        "disaster": lambda: q_disaster(strict),
        "reset": lambda: q_reset()
    }.get(filter_type, lambda: q_reset())()


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__, suppress_callback_exceptions=True)

# Serve logo from /assets folder (auto-detected by Dash)
logo_img_tag = html.Img(
    src="/assets/grazioso_logo.png",
    style={"height":"60px","marginRight":"12px"}
)

filter_controls = html.Div(
    style={"display":"flex","gap":"1rem","alignItems":"center","flexWrap":"wrap"},
    children=[
        html.Div(logo_img_tag) if logo_img_tag else html.Div(),
        html.Div([ html.H2("Grazioso Salvare: Search-and-Rescue Dashboard", style={"margin":"0"}),
                   html.Small("Built by Lynneska Rivera", style={"opacity":0.7}) ]),
        html.Div(style={"flex":"1"}),
        dcc.RadioItems(
            id='filter-type',
            options=[{"label":"Water Rescue","value":"water"},
                     {"label":"Mountain/Wilderness","value":"mountain"},
                     {"label":"Disaster/Tracking","value":"disaster"},
                     {"label":"Reset (All)","value":"reset"}],
            value="reset",
            labelStyle={"display":"inline-block","marginRight":"12px"}
        )
    ]
)

app.layout = html.Div([
    html.Div(filter_controls),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX":"auto"},
        style_cell={"minWidth":"100px","width":"140px","maxWidth":"260px","whiteSpace":"normal","textAlign":"left"},
        style_header={"fontWeight":"bold"},
        style_data_conditional=[]
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display':'flex','gap':'1rem'}, children=[
        html.Div(id='graph-id', className='col s12 m6', style={"flex":"1"}),
        html.Div(id='map-id',   className='col s12 m6', style={"flex":"1"})
    ])
])


#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id','data'), Input('filter-type','value'))
def update_dashboard(filter_type):
    dff = read_df(build_query(filter_type, strict=True))
    if dff.empty and filter_type in ("water","mountain","disaster"):
        dff = read_df(build_query(filter_type, strict=False))
    if dff.empty:
        dff = read_df(q_reset())
    return dff.to_dict('records')

@app.callback(Output('graph-id',"children"),
              Input('datatable-id',"derived_virtual_data"),
              State('datatable-id','data'))
def update_graphs(viewData, data_fallback):
    data_src = viewData if viewData is not None else data_fallback
    if not data_src:
        return [html.Div("No data to chart.")]
    dff = pd.DataFrame.from_dict(data_src)
    breed_col = RESOLVED["breed"]
    if breed_col is None or breed_col not in dff.columns:
        return [html.Div("Breed column not found; cannot render chart.")]
    result_count = html.Div(f"Results: {len(dff)}", style={"opacity":0.7,"marginBottom":"6px"})
    top = (dff[breed_col].fillna("(unknown)").astype(str).str.strip()
           .value_counts().head(15).reset_index()
           .rename(columns={"index":"breed", breed_col:"count"}))
    fig = px.bar(top, x="breed", y="count", title="Breed Distribution (Top 15)")
    fig.update_layout(xaxis_title="", yaxis_title="Count", margin=dict(l=10,r=10,t=50,b=80))
    fig.update_xaxes(tickangle=45)
    return [result_count, dcc.Graph(figure=fig)]

@app.callback(Output('map-id',"children"),
              Input('datatable-id',"derived_virtual_data"),
              Input('datatable-id',"derived_virtual_selected_rows"))
def update_map(viewData, selected_rows):
    if not viewData:
        return [html.Div("No data to map.")]
    dff = pd.DataFrame.from_dict(viewData)
    lat_col, lon_col = RESOLVED["lat"], RESOLVED["lon"]
    name_col = RESOLVED["name"] or (dff.columns[0] if len(dff.columns) else None)
    breed_col = RESOLVED["breed"]
    row_idx = (selected_rows or [0])[0]
    row_idx = max(0, min(row_idx, len(dff)-1))
    center_lat, center_lon = 30.75, -97.48
    marker_lat = dff.at[row_idx, lat_col] if lat_col in dff.columns else center_lat
    marker_lon = dff.at[row_idx, lon_col] if lon_col in dff.columns else center_lon
    tooltip_text = str(dff.at[row_idx, breed_col]) if breed_col in dff.columns else "Breed: (unknown)"
    popup_name = str(dff.at[row_idx, name_col]) if name_col in dff.columns else "(unknown)"
    try:
        lat = float(marker_lat); lon = float(marker_lon)
        if not np.isfinite(lat) or not np.isfinite(lon): lat, lon = center_lat, center_lon
    except Exception:
        lat, lon = center_lat, center_lon
    return [ dl.Map(style={'width':'100%','height':'500px'}, center=[lat,lon], zoom=11, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[lat,lon], children=[
                    dl.Tooltip(tooltip_text),
                    dl.Popup([html.H4("Animal Name"), html.P(popup_name)])
                ])
            ]) ]

app.run_server(debug=True)


Initial load (q_reset) rows: 16763
Dash app running on http://127.0.0.1:28356/


In [ ]:
print("DB:", db.database.name)
print("Collection:", db.collection.name)
print("Docs:", db.collection.count_documents({}))
doc = db.collection.find_one({})
print("Sample keys:", list(doc.keys())[:12])



In [ ]:
import os
os.path.exists("grazioso_logo.png")

